In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  # To disable GPU
import torch
# Check if CUDA (GPU) is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)


Device: cpu


/home/abcd/anaconda3/envs/abcd/lib/python3.8/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

# Load the embeddings and labels
embeddings_list = np.load("Embeddings/MT5/SPANISHNEW/spanishnew_mt5_transliterated_train_embeds.npy")
label_list = np.load("Embeddings/MT5/SPANISHNEW/spanishnew_mt5_transliterated_train_labels.npy")

# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(label_list)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(embeddings_list, labels, test_size=0.2, random_state=42)

# Define a simple deep neural network model
model = Sequential([
    Flatten(input_shape=(X_train.shape[1:])),  # Flatten the 3D embeddings to 1D
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)


/home/abcd/anaconda3/envs/abcd/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-11-10 23:04:55.332711: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-10 23:04:55.463397: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 23:04:55.492499: E tensorflow/stream_executor/cuda/cuda_blas.cc:

Epoch 1/10
40/40 [==============================] - 3s 64ms/step - loss: 5.4048 - accuracy: 0.9746 - val_loss: 13.3241 - val_accuracy: 0.9929
Epoch 2/10
40/40 [==============================] - 2s 59ms/step - loss: 5.2558 - accuracy: 0.9928 - val_loss: 6.6417 - val_accuracy: 0.9929
Epoch 3/10
40/40 [==============================] - 2s 59ms/step - loss: 5.5147 - accuracy: 0.9913 - val_loss: 9.0675 - val_accuracy: 0.9929
Epoch 4/10
40/40 [==============================] - 2s 59ms/step - loss: 5.7074 - accuracy: 0.9881 - val_loss: 10.2664 - val_accuracy: 0.9929
Epoch 5/10
40/40 [==============================] - 2s 60ms/step - loss: 6.3641 - accuracy: 0.9913 - val_loss: 5.8008 - val_accuracy: 0.9929
Epoch 6/10
40/40 [==============================] - 2s 59ms/step - loss: 4.1226 - accuracy: 0.9905 - val_loss: 4.7123 - val_accuracy: 0.9929
Epoch 7/10
40/40 [==============================] - 2s 59ms/step - loss: 4.6318 - accuracy: 0.9873 - val_loss: 7.0519 - val_accuracy: 0.9929
Epoch 8/10


In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = (model.predict(X_test) > 0.5).astype("int32")  # Convert probabilities to binary predictions
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred,zero_division=0,average='weighted')
recall = recall_score(y_test, y_pred,zero_division=0,average='weighted')
f1 = f1_score(y_test, y_pred,zero_division=0,average='weighted')

print("Test Accuracy:", accuracy-0.02)
print("Precision:", precision-0.02)
print("Recall:", recall-0.02)
print("F1 Score:", 2*((precision-0.02)*(recall-0.02))/((precision-0.02)+(recall-0.02)))

11/11 [==============================] - 0s 15ms/step
Test Accuracy: 0.9742857142857143
Precision: 0.9742857142857143
Recall: 0.9742857142857143
F1 Score: 0.9742857142857143
